In [ ]:
#abrir os dados
import pandas as pd

dados = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/iris.csv', sep=';')

In [ ]:
#balancear os dados
from collections import Counter
from imblearn.over_sampling import SMOTE

#instanciar o balanceador
resampler = SMOTE()

#segmentar os dados em atributos e classes
dados_atributos = dados.drop(columns=['class']) #somente atributos
dados_classes = dados['class'] #somente classes

#balancear
dados_atributos_balanceados, dados_classes_balanceados = resampler.fit_resample(dados_atributos, dados_classes)

In [ ]:
#normalizar
#separar coluna numerica da categorica, base ja separou
from sklearn import preprocessing
from pickle import dump

dados_num = dados_atributos_balanceados
dados_cat = dados_classes_balanceados

#instanciar normalizador
normalizador = preprocessing.MinMaxScaler()

#treinar o normalizador
modelo_normalizador_iris = normalizador.fit(dados_num)

#salvar normalizador
dump(modelo_normalizador_iris, open('/content/drive/MyDrive/SistemasInteligentes/models/normalizador_iris.model', 'wb'))

In [ ]:
#abrir modelo normalizador
from pickle import load

modelo_normalizador_iris = load(open('/content/drive/MyDrive/SistemasInteligentes/models/normalizador_iris.model', 'rb'))

#normalizar os dados
dados_num_normalizados = modelo_normalizador_iris.transform(dados_num)

dados_num_normalizados

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

In [ ]:
#normalizar os dados categoricos
dados_cat_normalizados = pd.get_dummies(dados_cat, dtype='int')

dados_cat_normalizados

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [ ]:
#juntar os dois fragmentos de dados em um dataframe
#converter dados num normalizado em dataframe
dados_num_normalizados = pd.DataFrame(dados_num_normalizados, columns=dados_atributos.columns)

dados = dados_num_normalizados.join(dados_cat_normalizados)

dados

,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,0.222222,0.625000,0.067797,0.041667,1,0,0
1,0.166667,0.416667,0.067797,0.041667,1,0,0
2,0.111111,0.500000,0.050847,0.041667,1,0,0
3,0.083333,0.458333,0.084746,0.041667,1,0,0
4,0.194444,0.666667,0.067797,0.041667,1,0,0
...,...,...,...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667,0,0,1
146,0.555556,0.208333,0.677966,0.750000,0,0,1
147,0.611111,0.416667,0.711864,0.791667,0,0,1
148,0.527778,0.583333,0.745763,0.916667,0,0,1


In [ ]:
#hiperparametrização ou numero de clusters
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
from scipy.spatial.distance import cdist #determinador de distancias
import numpy as np

distortions = [] #retorno de distorcoes
K = range(1,dados.shape[0]) #intervalo de clusters analisados para o numero otimo

for k in K:
  clusterModel = KMeans(n_clusters=k, random_state=42)
  clusterModel.fit(dados)
  distortions.append(sum(np.min(cdist(dados, clusterModel.cluster_centers_, 'euclidean'), axis=1)) / dados.shape[0])

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (147) found smaller than n_clusters (148). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (147) found smaller than n_clusters (149). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [ ]:
#grafico das distorcoes
fig, ax = plt.subplots()
ax.plot(K, distortions)
ax.set(xlabel='Numero de clusters', ylabel='Distorcao', title='Metodo Elbow')
ax.grid()
plt.show

In [ ]:
#determinar o numero de clusters
x0 = K[0]
y0 = distortions[0]

xn = K[-1]
yn = distortions[-1]
distancias = []

for i in range(len(distortions)):
  x = K[i]
  y = distortions[i]
  numerador = abs((yn-y0)*x-(xn-x0)*y+(xn*y0)-(yn*x0))
  denominador = math.sqrt((yn-y0)**2 + (xn-x0)**2)
  distancias.append(numerador/denominador)

numero_clusters_otimo = K[distancias.index(max(distancias))]

#treinar e salvar o modelo definitivo
iris_cluster_model = KMeans(n_clusters=numero_clusters_otimo, random_state=42)
iris_cluster_model.fit(dados)

#salvar o modelo
dump(iris_cluster_model, open('/content/drive/MyDrive/SistemasInteligentes/models/iris_clusters.model', 'wb'))